## Overview of Basic Monte Carlo Methods
### Ying Wai Li and Matthew Wilson (Los Alamos National Laboratory)

### 1. Equilibrium statistical mechanics at a glance

* A physical system has many ways to arrange itself $\rightarrow$ a configuration/state $\boldsymbol{x}$
* Different configurations correspond to different total energies $E$ $(=E(\boldsymbol{x}))$.

**Canonical ensemble**: system with a heat bath at fixed temperature ($T$), number of particles ($N$), volume ($V$) &rarr; ("$NVT$" ensemble)

**Canonical/Boltzmann distribution:**

Given temperature $T$, what is the probability of finding the system at state $\boldsymbol{x}$?
$$ P(\boldsymbol{x},T) = \frac{1}{Z(T)} e^{-E(\boldsymbol{x})/kT} $$

How do the total energies $E$ of the system distribute?
$$ P(E,T) = \frac{1}{Z(T)} g(E) e^{-E/kT}$$

where $Z(T)$ is the partition function:
$$Z(T) = \sum_{\boldsymbol{x}} e^{-E(\boldsymbol{x})/kT} = \sum_{E} g(E) e^{-E(\boldsymbol{x})/kT}$$

$k$: Boltzmann constant\
$g(E)$: "density of states" that counts the energy degeneracy

<p align="center">
  <img src="Figures/Boltzmann_distribution.png" width="500"/>
</p>

**Internal energy $U(T)$ of the system at $T$:**
$$U(T) = \langle E \rangle _T = \sum_E E \cdot P(E,T) = \frac{1}{Z(T)} \sum_E E g(E) e^{-E/kT} $$ 

where the angle brackets $\langle ... \rangle$ denote the ensemble average over the canonical ensemble.

**Heat capacity $C_V(T)$:**
$$C_V(T) = \frac{dU}{dT}$$

#### Connection between statistical mechanics and Monte Carlo (MC) simulations

* Total number of admissable states of a system can be more than astronomical\
(A 17 $\times$ 17 2D Ising model already has more states than the number of atoms in the universe = $10^{82}$!) 
* Monte Carlo (MC) sampling is a tool for sampling/selecting states according to a **desired probability distribution**
* Monte Carlo is best at studying **finite-temperature** equilibrium properties, phase transitions, and mapping phase diagrams
* For ground states optimization, choose your MC tools carefully! You probably want to use optimization techniques such as simulated annealing or parallel tempering. 

### 2. Metropolis sampling

* Generates a collection of states (energies) that obeys the canonical distribution at temperature $T$
* Metropolis sampling [J. Chem. Phys. **21**, 1087-1092 (1953)] is perhaps the most widely used importance sampling technique

<p align="center">
  <img src="Figures/Metropolis.png" width="800" />
</p>

**Algorithm:**

0. Prerequisites: choose a temperature $T$, number of MC steps to be performed, **equilibrate the system** (to be covered in MC practical).
1. Generate/propose a new configuration
2. Calculate the change in energy, $\Delta E = E(\boldsymbol{x'}) - E(\boldsymbol{x})$
3. Accept/reject with probability:
    $$ p_{acc}(\boldsymbol{x}\rightarrow\boldsymbol{x'}) = \min\left(1, \frac{P(E(\boldsymbol{x'}), T)}{P(E(\boldsymbol{x}), T)}\right) = \min\left(1, e^{-\Delta E/(k_BT)}\right). $$

    What is *really* happening at this step in a computer program?
    * If $e^{-\Delta E/(k_BT)} > 1$, accept the proposal $\boldsymbol{x'}$ right away
    * Otherwise, generate a random number $r \in [0,1)$
    * If $r > e^{-\Delta E/(k_BT)}$, accept the proposal
    * Otherwise, reject the proposal. **Keep the old state $\boldsymbol{x}$ and count it again.**

4. Calculate and accumulate all physical observables of interest (e.g. energy $E$, magnetization $M$, etc.)
5. Repeat steps 1-4 until a desired number of MC steps have been performed

<p align="center">
  <img src="Figures/MarkovChain.png" width="800"/>
</p>

#### Data post-processsing: calculate ensemble averages of physical observables

Assuming $N$ **statistically independent** samples are collected from the Metropolis simulation at temperature $T$, one can calculate different physical observables as follows:

Internal energy: 
$$U(T) = \langle E \rangle _T = \frac{1}{N} \sum_i^N E_i $$

Magnetization:
$$M(T) = \langle M \rangle _T = \frac{1}{N} \sum_i^N M_i $$

For any observable $\mathcal{O}$ in general:
$$ \langle \mathcal{O} \rangle _T = \frac{1}{N} \sum_i^N \mathcal{O}_i $$ 

Heat capacity:
$$C_V(T) = \frac{\langle E^2 \rangle - \langle E \rangle^2}{kT^2}$$

Magnetic susceptability:
$$\chi(T) = \frac{\langle M^2 \rangle - \langle M \rangle^2}{kT}$$

... and all other order parameters you would like to calculate!

*Questions:*
1. The Boltzmann distribution term $P(E,T)$ disappears in the equations. Why?
2. How to generate statistically independent samples?



### 3. Statistical analysis

#### 3.1 Basics
Given a set of $N$ **statistically independent** measurements of observable $\mathcal{O} = \{ \mathcal{O}_1, \mathcal{O}_2, ...,  \mathcal{O}_i, ..., \mathcal{O}_N \}$,

<p align="center">
  <img src="Figures/dataset.png" width="600"/>
</p>

Mean: 
$$\overline{\mathcal{O}} = \left< \mathcal{O} \right> = \frac{1}{N}\sum_i^N \mathcal{O}_i$$

Variance: 
$$\sigma(\mathcal{O})^2 = \frac{1}{N} \sum_i^N (\mathcal{O}_i - \overline{\mathcal{O}})^2 = \left< \mathcal{O}^2 \right> - \left< \mathcal{O} \right>^2 $$
[$\sigma(\mathcal{O})$: standard deviation]

Variance of the mean: 
$$ \sigma(\overline{\mathcal{O}})^2 = \frac{\sigma(\mathcal{O})^2}{N-1}$$

#### 3.2 Binning analysis: 

What if we would like to estimate the errors for a quantity that is not "measured" directly at each Monte Carlo step (e.g. the heat capacity $C_V$ is calculated from the energy fluctuations measured throughput the simulation)?

**Blocking:**
* Divide the whole data set into $N_b$ blocks, each has $N / N_b$ measurements
* Calculate the observable of interest (e.g. $C_V$) for each block $j$, denoted by $\mathcal{O}_{b,j}$

<p align="center">
  <img src="Figures/blocking.png" width="700"/>
</p>

* Mean and variance of the quantity can be calculated like before: 
$$ \overline{\mathcal{O}_b} = \frac{1}{N_b}\sum_j^{N_b} \mathcal{O}_{b,j} $$
$$ \sigma(\mathcal{O}_b)^2 = \frac{1}{N_b} \sum_j^{N_b} (\mathcal{O}_{b,j} - \overline{\mathcal{O}_b})^2 $$

**Bootstrapping/resampling:**
* Construct $M$ sets of data, each has $N$ measurements sampled from the original data set $\mathcal{O}$. Denote each set as $\mathcal{O}_m$ where $m = 1, 2, ..., M$. 
* Possible to have a measurement appearing multiple times in the resampled sets.

<p align="center">
  <img src="Figures/bootstrapping.png" width="700"/>
</p>

* Mean of the measurements for each set:
$$\overline{\mathcal{O_m}} = \frac{1}{N}\sum_i^N \mathcal{O}_{m,i}$$
* Mean of the means to be the estimator of that observable:
$$\overline{\mathcal{O}} = \frac{1}{M}\sum_j^M \overline{\mathcal{O}_m} $$
* Error estimation:
$$\sigma(\overline{\mathcal{O}})^2 = \overline{\mathcal{O}^2} - \overline{\mathcal{O}}^2 $$


**Jackknife analysis:**
* The "leave-one-out" method; similar to cross validation in machine learning
* Construct a new, reduced data set by leaving out one measurement from the full data set $\mathcal{O}$
* There are $N$ options to do so, this yields $N$ new data sets each with $N-1$ measurements.
* Mean and variance can be calculated similarly. 

<p align="center">
  <img src="Figures/jackknife.png" width="700"/>
</p>

### 4. Autocorrelation function and correlation time

The correlation time gives us an idea of the typical time scale for a state to transition to another significantly different (uncorrelated) state. (Think about the half-life of radioactive decay.) This is useful for us to obtain statistically independent samples.

#### 4.1 Definitions

**Time-displaced autocorrelation for an observable $\mathcal{O}$:**

$$
\begin{aligned}
\mathcal{A}(t) &= \int dt' \left[ \mathcal{O}(t') - \langle \mathcal{O} \rangle \right] \left[ \mathcal{O}(t'+ t) - \langle \mathcal{O} \rangle \right] \\
               &= \int dt' \left[ \mathcal{O}(t')\mathcal{O}(t'+ t) - \langle \mathcal{O} \rangle^2 \right]
\end{aligned}
$$

**Autocorrelation function (discretized and normalized with the observables's fluctuation):**

 $$ \mathcal{A}(t) = \frac{\left<\mathcal{O}_i\mathcal{O}_{i+t}\right> - \left<\mathcal{O}_i\right>^2}{\left<\mathcal{O}_i^2\right> - \left<\mathcal{O}_i\right>^2} $$

[TODO: will add a picture illustration]

For example, to calculate $\left<\mathcal{O}_i\mathcal{O}_{i+2}\right>$:

<p align="center">
  <img src="Figures/correlation_function.png" width="500"/>
</p>



**Correlation time**

* The autocorrelation function $\mathcal{A}(t)$ is expected to decay exponentially at long times, hence $\mathcal{A}(t) \sim e^{-t/\tau}$ for $t \rightarrow \infty$
* $\tau$ is defined as the correlation time

<p align="center">
  <img src="Figures/correlation_time.png" width="500"/>
</p>

**Integrated correlation time**

If, the variance of the mean is proven to be: 
    $$ \sigma(\overline{\mathcal{O}})^2 = \frac{\sigma(\mathcal{O})^2}{N} \left[ 1 + 2 \sum_t^{N-1} \left( 1 - \frac{t}{N} \right) a(t) \right], $$ 

where $a(t) = \mathcal{A}(t) / \mathcal{A}(0)$.

The integrated correlation time is defined as (in discrete form):

$$\tau_{\rm int} =  1 + 2 \sum_t^{N-1} \left( 1 - \frac{t}{N} \right) a(t)$$

In the long time limit of $N \rightarrow \infty$,

$$\tau_{\rm int} = 1 + 2 \sum_t^{\infty} a(t) $$


#### 4.2 Implementation details

* The calculation of autocorrelation function has an $\mathcal{O}(N^2)$ computational complexity, where $N$ is the number of samples. ($\mathcal{O}$ as for the [Big-O notation](https://en.wikipedia.org/wiki/Big_O_notation), not to be confused with the observable symbol $\mathcal{O}$.) 
* Assume that the samples are a series of $N$ evenly-spaced measurements, a trick to compute autocorrelation function is by using fast Fourier transform (FFT) to calculate the Fourier transform $\tilde{\mathcal{A}}(\omega)$ of the autocorrelation and then inverting the Fourier transform to get $\mathcal{A}(t)$ back. This has an $\mathcal{O}(N \log N)$ computational complexity.

    $$
    \begin{aligned}
    \tilde{\mathcal{A}}(\omega) &= \int dt e^{-i \omega t} \mathcal{A}(t) \\
                                &= \int dt e^{-i \omega t} \int dt' \left[ \mathcal{O}(t') - \langle \mathcal{O} \rangle \right] \left[ \mathcal{O}(t'+ t) - \langle \mathcal{O} \rangle \right] \\
                                &= \int dt \int dt' e^{i \omega t'} \left[ \mathcal{O}(t') - \langle \mathcal{O} \rangle \right]  e^{-i \omega (t'+t)} \left[ \mathcal{O}(t'+ t) - \langle \mathcal{O} \rangle \right] \\
                                &= \tilde{\mathcal{O}'}(\omega) \tilde{\mathcal{O}'}(-\omega) \\
                                &= \left| \tilde{\mathcal{O}'}(\omega) \right|^2,
    \end{aligned}
    $$
    where $\tilde{\mathcal{O}'}(\omega)$ is the Fourier transform of $\mathcal{O}'(t) \equiv \mathcal{O}(t) - \langle \mathcal{O} \rangle$.

* So we can first calculate $\tilde{\mathcal{O}'}(\omega)$ (the Fourier transform of the "time series" of observable $\mathcal{O}(t)$ shifted by a constant $\langle \mathcal{O} \rangle$), get $\tilde{\mathcal{A}}(\omega)$, then calculate $\mathcal{A}(t)$ by:

$$
\mathcal{A}(t) = \int d\omega e^{i \omega t} \tilde{\mathcal{A}}(\omega).
$$

#### 4.3 A few notes on practicals:
* The autocorrelation function has different "stages"
    - (I): At short time $t$, combination of multiple correlation times at play
    - (II): rather exponential, one of the characteristic correlation time dominates
    - (III): At large time $t$, statistical noise dominates
    
    $\rightarrow$ choose the region carefully when fitting the correlation time
<p align="center">
  <img src="Figures/correlation_stages.png" width="500"/>
</p>

* Different physical quantities have different correlation times (e.g. $E$ vs $M$)
* The calculation of integrated correlation time is often more stable than the (exponential) correlation time
* Samples drawn at intervals of $2 \tau$ present reasonable statistical independence
* It can be proven that the integrated correlation time $\tau_{\rm int}$ is roughly $2 \tau$
* Correlation time is different at different temperature $T$. It can be extremely long around transition temperature (e.g. critical slowing down around second order phase transition)

#### 5. Finite size effects

1. Different system sizes you put in the MC simulations will give you qualitatively different results.

    e.g. Parallel tempering simulation on 2D ferromagnetic Ising model of different system sizes

<p align="center">
  <img src="Figures/ising_finite_size.png" width="900"/>
</p>
